In [ ]:
# Set up Phoenix Tracing
import nest_asyncio

nest_asyncio.apply()

import os
from pathlib import Path
from dotenv import load_dotenv

from phoenix.client import Client
from phoenix.client.types.spans import SpanQuery
import pandas as pd

from phoenix.otel import register

from openinference.instrumentation.openai import GoogleGenInstrumentor



pd.set_option('display.max_columns', 500)



/Users/user1/AI/AI_Tutorial/GenAI_Tutorial/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load environment variables from local environment

load_dotenv(Path('../../.env'))
os.environ["PHOENIX_API_KEY"] = os.getenv("PHOENIX_API_KEY")
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = os.getenv("PHOENIX_COLLECTOR_ENDPOINT")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [ ]:
from google.colab import userdata
os.environ["PHOENIX_API_KEY"] = userdata.get('PHOENIX_API_KEY')
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = userdata.get('PHOENIX_COLLECTOR_ENDPOINT')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

## Creating Traces for the Evaluation Task

In [39]:
project_name = "calculus-code-agent-eval"  # project for our application traces
tracer_provider = register(project_name=project_name,
                        auto_instrument=True
                            )





/Users/user1/AI/AI_Tutorial/GenAI_Tutorial/.venv/lib/python3.12/site-packages/phoenix/otel/otel.py:333: UserWarning: Could not infer collector endpoint protocol, defaulting to HTTP.
  warnings.warn("Could not infer collector endpoint protocol, defaulting to HTTP.")
Overriding of current TracerProvider is not allowed


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: calculus-code-agent-eval
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/s/comradedaniel/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'authorization': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [4]:
# Define the project identifier
project_identifier = "calculus-code-agent"

# Export all the top level spans 
spansdf = Client().spans.get_spans_dataframe(project_identifier=project_identifier)

spansdf.head()

,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id,attributes.llm.token_count.total,attributes.output.mime_type,attributes.llm.provider,attributes.llm.output_messages,attributes.input.value,attributes.llm.model_name,attributes.llm.system,attributes.llm.token_count.prompt,attributes.llm.invocation_parameters,attributes.input.mime_type,attributes.output.value,attributes.metadata,attributes.openinference.span.kind,attributes.llm.input_messages,attributes.llm.token_count.completion,attributes.tool.name,attributes.tool.description,attributes.llm.function_call,attributes.llm.tools,attributes.llm.prompt_template.template
context.span_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
689f9682f55aac95,ChatGoogleGenerativeAI,LLM,03d8e3818ef8ab51,2025-10-04 14:00:28.327351+00:00,2025-10-04 14:00:29.109825+00:00,OK,,[],689f9682f55aac95,5236e0e895c8beec6c6de099d18a03cb,253.0,application/json,google,"[{'message.role': 'assistant', 'message.conten...","{""messages"": [[{""lc"": 1, ""type"": ""constructor""...",gemini-2.0-flash-lite-001,vertexai,213.0,"{""model"": ""models/gemini-2.0-flash-lite-001"", ...",application/json,"{""generations"": [[{""text"": ""The directional de...","{'ls_provider': 'google_genai', 'checkpoint_ns...",LLM,"[{'message.role': 'user', 'message.content': '...",40.0,None,None,None,None,None
03d8e3818ef8ab51,generate_answer,CHAIN,d014d38edd9e596e,2025-10-04 14:00:28.326253+00:00,2025-10-04 14:00:29.420738+00:00,OK,,[],03d8e3818ef8ab51,5236e0e895c8beec6c6de099d18a03cb,NaN,None,None,None,"{""question"": ""Compute the directional derivati...",None,None,NaN,None,application/json,"The directional derivative of f(x, y) = x^2y a...","{'langgraph_node': 'generate_answer', 'langgra...",CHAIN,None,NaN,None,None,None,None,None
fb9fd158550a42f1,execute_python_code,TOOL,950dfd5eaf785914,2025-10-04 14:00:27.777778+00:00,2025-10-04 14:00:27.786548+00:00,OK,,[],fb9fd158550a42f1,5236e0e895c8beec6c6de099d18a03cb,NaN,None,None,None,"\nimport sympy\n\nx, y = sympy.symbols('x y')\...",None,None,NaN,None,None,Code executed successfully\n\n**Standard Outpu...,{'checkpoint_ns': 'run_calculus_code:5dec090e-...,TOOL,None,NaN,execute_python_code,Execute Python code using CodeInterpreter and ...,None,None,None
950dfd5eaf785914,run_calculus_code,CHAIN,d014d38edd9e596e,2025-10-04 14:00:27.777121+00:00,2025-10-04 14:00:28.052805+00:00,OK,,[],950dfd5eaf785914,5236e0e895c8beec6c6de099d18a03cb,NaN,None,None,None,"{""question"": ""Compute the directional derivati...",None,None,NaN,None,application/json,Code executed successfully\n\n**Standard Outpu...,"{'langgraph_node': 'run_calculus_code', 'langg...",CHAIN,None,NaN,None,None,None,None,None
a359ae3e50d656bc,JsonOutputKeyToolsParser,UNKNOWN,f2287d6c9b6fddd9,2025-10-04 14:00:26.959655+00:00,2025-10-04 14:00:26.960364+00:00,OK,,[],a359ae3e50d656bc,5236e0e895c8beec6c6de099d18a03cb,NaN,None,None,None,"{""content"": """", ""additional_kwargs"": {""functio...",None,None,NaN,None,application/json,"\nimport sympy\n\nx, y = sympy.symbols('x y')\...",{'checkpoint_ns': 'write_calculus_code:e3ba1df...,UNKNOWN,None,NaN,None,None,None,None,None


Looking at the traces in `spansdf`, you will see lots of parameters in the columns. The most important of of them are: `name`, `context.span_id`, `context.trace_id`, `attributes.input.value`, `attributes.output.value`.

In the next cell, we will use `SpansQuery()` to fetch attributes related to `write_calculus_code`, `execute_python_code` and `generate_answer`.

In [5]:
# Extract the question and generated code from the spans
query = SpanQuery().where("name == 'write_calculus_code'")
spansdf = Client().spans.get_spans_dataframe(query=query, project_identifier=project_identifier)

# drop any traces not from our dataset
spansdf.dropna(
    subset=["attributes.metadata"], inplace=True
)

spansdf["question"] = spansdf["attributes.input.value"] 
spansdf["code"] = spansdf["attributes.output.value"]

# Now get the code execution result spans
query = SpanQuery().where("name == 'execute_python_code'")
code_res_df = Client().spans.get_spans_dataframe(
    query=query, project_identifier=project_identifier
)

code_res_df["result"] = code_res_df["attributes.output.value"]

# Now get the answer generation spans
query = SpanQuery().where("name == 'generate_answer'")
gen_df = Client().spans.get_spans_dataframe(
    query=query, project_identifier=project_identifier
)

gen_df["answer"] = gen_df["attributes.output.value"]


# Combine the spans together 
spans_df = spansdf.merge(
    code_res_df[["context.trace_id", "result",]],
    on="context.trace_id",
    how="left",
)

spans_df = spans_df.merge(
    gen_df[["context.trace_id", "answer",]],
    on="context.trace_id",
    how="left",
)

spans_df.head()

,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id,attributes.input.value,attributes.openinference.span.kind,attributes.output.value,attributes.metadata,question,code,result,answer
0,write_calculus_code,CHAIN,d014d38edd9e596e,2025-10-04 14:00:24.650945+00:00,2025-10-04 14:00:27.505888+00:00,OK,,[],97b824be1c4637d0,5236e0e895c8beec6c6de099d18a03cb,"Compute the directional derivative of f(x, y) ...",CHAIN,"\nimport sympy\n\nx, y = sympy.symbols('x y')\...","{'langgraph_node': 'write_calculus_code', 'lan...","Compute the directional derivative of f(x, y) ...","\nimport sympy\n\nx, y = sympy.symbols('x y')\...",Code executed successfully\n\n**Standard Outpu...,"The directional derivative of f(x, y) = x^2y a..."
1,write_calculus_code,CHAIN,38d37c204900e73e,2025-10-04 13:59:59.278405+00:00,2025-10-04 14:00:01.483023+00:00,OK,,[],6c299cbb03ab06ee,8b2cb4bb8ae6f59b98d54ccff051d3b3,Evaluate the definite integral from -∞ to ∞ of...,CHAIN,"import sympy\nfrom sympy import integrate, oo,...","{'langgraph_node': 'write_calculus_code', 'lan...",Evaluate the definite integral from -∞ to ∞ of...,"import sympy\nfrom sympy import integrate, oo,...",Code execution failed\n\n**Error Log:**\n```\n...,The code failed because the variable `x` was n...
2,write_calculus_code,CHAIN,2bcee93de19f0fe5,2025-10-04 13:59:34.613106+00:00,2025-10-04 13:59:36.917160+00:00,OK,,[],95f7d951b948784e,2979a98f3de7f64a15fc6248e82e9f61,Find the Taylor polynomial of order 3 for f(x)...,CHAIN,"import sympy\nfrom sympy import cos, series, S...","{'langgraph_node': 'write_calculus_code', 'lan...",Find the Taylor polynomial of order 3 for f(x)...,"import sympy\nfrom sympy import cos, series, S...",Code executed successfully\n\n**Standard Outpu...,The Taylor polynomial of order 3 for f(x) = co...
3,write_calculus_code,CHAIN,cd979c8f682ca58b,2025-10-04 13:59:07.968526+00:00,2025-10-04 13:59:11.661038+00:00,OK,,[],4a5711326795cb3e,5f0fd2066a1d357c9be4cd724e3ddc70,Use Green's theorem to evaluate ∮C (y dx - x d...,CHAIN,"import sympy\nfrom sympy import integrate, sin...","{'langgraph_node': 'write_calculus_code', 'lan...",Use Green's theorem to evaluate ∮C (y dx - x d...,"import sympy\nfrom sympy import integrate, sin...",Code executed successfully\n\n**Standard Outpu...,The user asked to evaluate the line integral ∮...
4,write_calculus_code,CHAIN,b4a67a0cafb0969f,2025-10-04 13:58:26.498836+00:00,2025-10-04 13:58:45.578624+00:00,OK,,[],df3ca0c226116a46,540176bfb532de742bc2696ec49a9f69,"Evaluate the line integral ∫C (x dx + y dy), w...",CHAIN,\nimport sympy\n\nx = sympy.Symbol('x')\ny = s...,"{'langgraph_node': 'write_calculus_code', 'lan...","Evaluate the line integral ∫C (x dx + y dy), w...",\nimport sympy\n\nx = sympy.Symbol('x')\ny = s...,Code executed successfully\n\n**Standard Outpu...,"The line integral ∫C (x dx + y dy), where C is..."


### Evaluation 1: Hallucination

We will use a higher Google LLM `gemini-2.0-flash-001` model as the LLM as a Judge

In [6]:
from phoenix.evals import (
    HALLUCINATION_PROMPT_RAILS_MAP,
    GoogleGenAIModel,
    llm_classify,
)


# API key will be read from environment
model = GoogleGenAIModel(model="gemini-2.0-flash-001", initial_rate_limit=0.1)

In [23]:
# Define the hallucination prompt template

HALLUCINATION_PROMPT_TEMPLATE = """
In this task, you will be given a query, code generated to address the query, 
the execution result of that code, and an answer produced from them. 
Your job is to check if the answer is faithful to the provided code and result.

- "factual" means the answer accurately reflects the information in the code and result.  
- "hallucinated" means the answer introduces details not supported by the code and result, 
  or contradicts them.  

Output exactly one word: either "factual" or "hallucinated". Do not output anything else.

[BEGIN DATA]
************
[Question]: {question}
************
[Generated code]: {code}
************
[Generated result]: {result}
************
[Answer]: {answer}
************
[END DATA]
"""


In [7]:
HALLUCINATION_PROMPT_TEMPLATE = """
In this task, you will be given a question, code generated to address the question, 
the execution result of that code, and an answer produced from them. 
Your job is to check if the answer is faithful to the provided code and result.

- "factual" means the answer accurately reflects the information in the code and result.  
- "hallucinated" means the answer introduces details not supported by the code and result, 
  or contradicts them.  

  Output exactly one word: either "factual" or "hallucinated".


  [BEGIN DATA]
  ************
  [Question]: {question}
  ************
  [Generated code]: {code}
  ************
  [Generated result]: {result}
  ************
  [Answer]: {answer}
  ************
  [END DATA]

  Write out a short EXPLANATION to show how to show that the answer is "factual" or "hallucinated". Avoid simply
  stating the correct answer at the outset. Your response LABEL should be a single word: either
  "factual" or "hallucinated", and it should not include any other text or characters. "hallucinated"
  indicates that the answer provides factually inaccurate information to the question based on the
   code and result. "factual" indicates that the answer to the question is correct relative to the
   code, and does not contain made up information.
"""

In [31]:
# Start the hallucination evaluation
rails = list(HALLUCINATION_PROMPT_RAILS_MAP.values())
hallucination_classifications = llm_classify(
    data=spans_df[:20], 
    template=HALLUCINATION_PROMPT_TEMPLATE, 
    model=model, 
    rails=rails,
    provide_explanation=True, #optional to generate explanations for the value produced by the eval LLM
    verbose=False
)

llm_classify |████      | 8/20 (40.0%) | ⏳ 00:12<00:33 |  2.75s/it 

Exception in worker on attempt 1: raised ClientError("429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.\\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 15\\nPlease retry in 9.695226749s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.0-flash'}, 'quotaValue': '15'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleap

llm_classify |████████  | 16/20 (80.0%) | ⏳ 00:19<00:01 |  2.25it/s 

Exception in worker on attempt 2: raised ClientError("429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.\\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 15\\nPlease retry in 2.170087796s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.0-flash'}, 'quotaValue': '15'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleap

llm_classify |████████  | 16/20 (80.0%) | ⏳ 00:23<00:01 |  2.25it/s 

...

llm_classify |████████  | 16/20 (80.0%) | ⏳ 00:58<00:01 |  2.25it/s 

Exception in worker on attempt 7: raised ClientError("429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.\\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 15\\nPlease retry in 22.855183622s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.0-flash'}, 'quotaValue': '15'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googlea

llm_classify |████████  | 16/20 (80.0%) | ⏳ 00:59<00:01 |  2.25it/s 

Exception in worker on attempt 6: raised ClientError("429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.\\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 15\\nPlease retry in 22.50183593s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.0-flash'}, 'quotaValue': '15'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleap

llm_classify |██████████| 20/20 (100.0%) | ⏳ 01:10<00:00 |  5.37s/it

In [9]:

hallucination_classifications.head()

,label,explanation,exceptions,execution_status,execution_seconds,prompt_tokens,completion_tokens,total_tokens
0,factual,The code calculates the directional derivative...,[],COMPLETED,2.336017,512,79,591
1,factual,The code failed because 'x' was not defined as...,[],COMPLETED,2.237872,617,61,678
2,factual,The code calculates the Taylor polynomial of o...,[],COMPLETED,8.487299,426,55,481
3,factual,The question asks to evaluate a line integral ...,[],COMPLETED,12.684271,666,66,732
4,factual,The code calculates the line integral of x dx ...,[],COMPLETED,12.315216,543,43,586


In [10]:
hallucination_classifications.label.value_counts()

label
factual         9
hallucinated    1
Name: count, dtype: int64

In [25]:
hallucination_classifications.rename(columns={"label": "hallucination_label", "explanation": "hallucination_explanation"}, inplace=True)
hallucination_classifications.head()

,hallucination_label,hallucination_explanation,exceptions,execution_status,execution_seconds,prompt_tokens,completion_tokens,total_tokens
0,factual,The code calculates the directional derivative...,[],COMPLETED,2.336017,512,79,591
1,factual,The code failed because 'x' was not defined as...,[],COMPLETED,2.237872,617,61,678
2,factual,The code calculates the Taylor polynomial of o...,[],COMPLETED,8.487299,426,55,481
3,factual,The question asks to evaluate a line integral ...,[],COMPLETED,12.684271,666,66,732
4,factual,The code calculates the line integral of x dx ...,[],COMPLETED,12.315216,543,43,586


## Checking for Code Readability

In [11]:
from phoenix.evals import CODE_READABILITY_PROMPT_RAILS_MAP, TOOL_CALLING_PROMPT_RAILS_MAP

In [12]:
# Define the code readability prompt template
CODE_READABILITY_PROMPT_TEMPLATE = """
    You will be given a question and a piece of code. 
    Your task is to judge whether the code is easy to read and understand.

    - "readable" means the code is clear, structured, and understandable.  
    - "unreadable" means the code is messy, confusing, or difficult to follow.  

    Output exactly one word: either "readable" or "unreadable".

    [BEGIN DATA]
    ************
    [Question]: {question}
    ************
    [Code]: {code}
    ************
    [END DATA]

    Output exactly one word: either "readable" or "unreadable".

    Please read the question, code, result and answer carefully, then write out in a step by step manner
    an EXPLANATION to show how to determine if the code is "readable" or "unreadable". Avoid simply
    stating the correct answer at the outset. Your response LABEL should be a single word: either
    "readable" or "unreadable", and it should not include any other text or characters.
"""


In [13]:
code_rails = list(CODE_READABILITY_PROMPT_RAILS_MAP.values())
code_readability = llm_classify(
    data=spans_df[:10], 
    template=CODE_READABILITY_PROMPT_TEMPLATE, 
    model=model, 
    rails=code_rails,
    provide_explanation=True,
)

llm_classify |██████████| 10/10 (100.0%) | ⏳ 01:06<00:00 |  6.64s/it


In [26]:
code_readability.rename(columns={"label": "code_readability_label", "explanation": "code_readability_explanation"}, inplace=True)
code_readability.head()

,code_readability_label,code_readability_explanation,exceptions,execution_status,execution_seconds,prompt_tokens,completion_tokens,total_tokens
0,readable,The code calculates the directional derivative...,[],COMPLETED,2.984788,381,194,575
1,readable,The code imports the necessary libraries (`sym...,[],COMPLETED,2.558920,276,70,346
2,readable,"First, analyze the question. The question is s...",[],COMPLETED,3.536373,299,247,546
3,readable,The code is well-structured and uses meaningfu...,[],COMPLETED,2.271392,453,96,549
4,readable,The code uses the `sympy` library to calculate...,[],COMPLETED,2.600496,419,102,521


In [15]:
code_readability['explanation'][0]

"The code calculates the directional derivative of a function. Let's check its readability.\n\n1.  **Imports:** It imports `sympy`, which is appropriate for symbolic calculations.\n2.  **Symbols:** It defines `x` and `y` as symbolic variables.\n3.  **Function Definition:** The function `f` is defined clearly.\n4.  **Gradient:** The gradient is computed using `f.diff()`.\n5.  **Point and Direction:** The point and direction are defined as matrices.\n6.  **Gradient at the Point:** `grad.subs()` is used to evaluate the gradient.\n7.  **Normalization:** The direction vector is normalized.\n8.  **Directional Derivative:** The dot product is calculated correctly.\n9.  **Output:** The result is printed.\n\nThe code is well-structured and easy to follow. Variable names are descriptive.\n\nLABEL: readable\n"

In [16]:
code_readability.label.value_counts()

label
readable    10
Name: count, dtype: int64

## Tool Calling

In [19]:
TOOL_CALLING_PROMPT_TEMPLATE = """
You will be given a question, code executed by the tool, and the result of that execution. 
Your task is to decide whether the tool call (code + result) correctly answers the question.

- "correct" means the code was appropriate for the question, the execution result is consistent with it, 
  and no extra information outside the question was introduced.  
- "incorrect" means the code does not match the question, produces a result unrelated to the question, 
  or introduces information not present in the question.  

[BEGIN DATA]
************
[Question]: {question}
************
[Code Executed]: {code}
************
[Result]: {result}
************
[END DATA]

Output exactly one word: either "correct" or "incorrect". 

Please read the question, code, result and answer carefully, then write out in a step by step manner
an EXPLANATION to show how to determine if the code is "correct" or "incorrect". Avoid simply
stating the correct answer at the outset. Your response LABEL should be a single word: either
"correct" or "incorrect", and it should not include any other text or characters.
"""


In [20]:
tool_rails = list(TOOL_CALLING_PROMPT_RAILS_MAP.values())

tool_call_evaluations = llm_classify(
    data=spans_df[:10], 
    template=TOOL_CALLING_PROMPT_TEMPLATE, 
    model=model, 
    rails=tool_rails,
    provide_explanation=True, #optional to generate explanations for the value produced by the eval LLM
)

llm_classify |██████████| 10/10 (100.0%) | ⏳ 00:12<00:00 |  1.22s/it

In [27]:
tool_call_evaluations.rename(columns={"label": "tool_call_label", "explanation": "tool_call_explanation"}, inplace=True)
tool_call_evaluations.head()

,tool_call_label,tool_call_explanation,exceptions,execution_status,execution_seconds,prompt_tokens,completion_tokens,total_tokens
0,correct,Explanation:\nThe question asks for the direct...,[],COMPLETED,3.753500,417,305,722
1,NOT_PARSABLE,The question asks to evaluate a definite integ...,[],COMPLETED,1.527971,418,73,491
2,correct,The question asks for the Taylor polynomial of...,[],COMPLETED,2.459969,338,99,437
3,correct,Explanation:\nThe question asks to evaluate th...,[],COMPLETED,2.532084,489,97,586
4,correct,Explanation:\nThe question asks to evaluate th...,[],COMPLETED,2.877039,452,170,622


In [29]:
spanseval = pd.concat([spans_df[:10], hallucination_classifications[['hallucination_label', 'hallucination_explanation']],
           code_readability[['code_readability_label', 'code_readability_explanation']],
           tool_call_evaluations[['tool_call_label', 'tool_call_explanation']]], axis=1)
spanseval.head()

,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id,attributes.input.value,attributes.openinference.span.kind,attributes.output.value,attributes.metadata,question,code,result,answer,hallucination_label,hallucination_explanation,code_readability_label,code_readability_explanation,tool_call_label,tool_call_explanation
0,write_calculus_code,CHAIN,d014d38edd9e596e,2025-10-04 14:00:24.650945+00:00,2025-10-04 14:00:27.505888+00:00,OK,,[],97b824be1c4637d0,5236e0e895c8beec6c6de099d18a03cb,"Compute the directional derivative of f(x, y) ...",CHAIN,"\nimport sympy\n\nx, y = sympy.symbols('x y')\...","{'langgraph_node': 'write_calculus_code', 'lan...","Compute the directional derivative of f(x, y) ...","\nimport sympy\n\nx, y = sympy.symbols('x y')\...",Code executed successfully\n\n**Standard Outpu...,"The directional derivative of f(x, y) = x^2y a...",factual,The code calculates the directional derivative...,readable,The code calculates the directional derivative...,correct,Explanation:\nThe question asks for the direct...
1,write_calculus_code,CHAIN,38d37c204900e73e,2025-10-04 13:59:59.278405+00:00,2025-10-04 14:00:01.483023+00:00,OK,,[],6c299cbb03ab06ee,8b2cb4bb8ae6f59b98d54ccff051d3b3,Evaluate the definite integral from -∞ to ∞ of...,CHAIN,"import sympy\nfrom sympy import integrate, oo,...","{'langgraph_node': 'write_calculus_code', 'lan...",Evaluate the definite integral from -∞ to ∞ of...,"import sympy\nfrom sympy import integrate, oo,...",Code execution failed\n\n**Error Log:**\n```\n...,The code failed because the variable `x` was n...,factual,The code failed because 'x' was not defined as...,readable,The code imports the necessary libraries (`sym...,NOT_PARSABLE,The question asks to evaluate a definite integ...
2,write_calculus_code,CHAIN,2bcee93de19f0fe5,2025-10-04 13:59:34.613106+00:00,2025-10-04 13:59:36.917160+00:00,OK,,[],95f7d951b948784e,2979a98f3de7f64a15fc6248e82e9f61,Find the Taylor polynomial of order 3 for f(x)...,CHAIN,"import sympy\nfrom sympy import cos, series, S...","{'langgraph_node': 'write_calculus_code', 'lan...",Find the Taylor polynomial of order 3 for f(x)...,"import sympy\nfrom sympy import cos, series, S...",Code executed successfully\n\n**Standard Outpu...,The Taylor polynomial of order 3 for f(x) = co...,factual,The code calculates the Taylor polynomial of o...,readable,"First, analyze the question. The question is s...",correct,The question asks for the Taylor polynomial of...
3,write_calculus_code,CHAIN,cd979c8f682ca58b,2025-10-04 13:59:07.968526+00:00,2025-10-04 13:59:11.661038+00:00,OK,,[],4a5711326795cb3e,5f0fd2066a1d357c9be4cd724e3ddc70,Use Green's theorem to evaluate ∮C (y dx - x d...,CHAIN,"import sympy\nfrom sympy import integrate, sin...","{'langgraph_node': 'write_calculus_code', 'lan...",Use Green's theorem to evaluate ∮C (y dx - x d...,"import sympy\nfrom sympy import integrate, sin...",Code executed successfully\n\n**Standard Outpu...,The user asked to evaluate the line integral ∮...,factual,The question asks to evaluate a line integral ...,readable,The code is well-structured and uses meaningfu...,correct,Explanation:\nThe question asks to evaluate th...
4,write_calculus_code,CHAIN,b4a67a0cafb0969f,2025-10-04 13:58:26.498836+00:00,2025-10-04 13:58:45.578624+00:00,OK,,[],df3ca0c226116a46,540176bfb532de742bc2696ec49a9f69,"Evaluate the line integral ∫C (x dx + y dy), w...",CHAIN,\nimport sympy\n\nx = sympy.Symbol('x')\ny = s...,"{'langgraph_node': 'write_calculus_code', 'lan...","Evaluate the line integral ∫C (x dx + y dy), w...",\nimport sympy\n\nx = sympy.Symbol('x')\ny = s...,Code executed successfully\n\n**Standard Outpu...,"The line integral ∫C (x dx + y dy), where C is...",factual,The code calculates the line integral of x dx ...,readable,The code uses the `sympy` library to calculate...,correct,Explanation:\nThe question asks to evaluate th...


In [32]:
spanseval.rename(columns={"hallucination_label": "label",}, inplace=True)
spanseval.head()

,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id,attributes.input.value,attributes.openinference.span.kind,attributes.output.value,attributes.metadata,question,code,result,answer,label,hallucination_explanation,code_readability_label,code_readability_explanation,tool_call_label,tool_call_explanation
0,write_calculus_code,CHAIN,d014d38edd9e596e,2025-10-04 14:00:24.650945+00:00,2025-10-04 14:00:27.505888+00:00,OK,,[],97b824be1c4637d0,5236e0e895c8beec6c6de099d18a03cb,"Compute the directional derivative of f(x, y) ...",CHAIN,"\nimport sympy\n\nx, y = sympy.symbols('x y')\...","{'langgraph_node': 'write_calculus_code', 'lan...","Compute the directional derivative of f(x, y) ...","\nimport sympy\n\nx, y = sympy.symbols('x y')\...",Code executed successfully\n\n**Standard Outpu...,"The directional derivative of f(x, y) = x^2y a...",factual,The code calculates the directional derivative...,readable,The code calculates the directional derivative...,correct,Explanation:\nThe question asks for the direct...
1,write_calculus_code,CHAIN,38d37c204900e73e,2025-10-04 13:59:59.278405+00:00,2025-10-04 14:00:01.483023+00:00,OK,,[],6c299cbb03ab06ee,8b2cb4bb8ae6f59b98d54ccff051d3b3,Evaluate the definite integral from -∞ to ∞ of...,CHAIN,"import sympy\nfrom sympy import integrate, oo,...","{'langgraph_node': 'write_calculus_code', 'lan...",Evaluate the definite integral from -∞ to ∞ of...,"import sympy\nfrom sympy import integrate, oo,...",Code execution failed\n\n**Error Log:**\n```\n...,The code failed because the variable `x` was n...,factual,The code failed because 'x' was not defined as...,readable,The code imports the necessary libraries (`sym...,NOT_PARSABLE,The question asks to evaluate a definite integ...
2,write_calculus_code,CHAIN,2bcee93de19f0fe5,2025-10-04 13:59:34.613106+00:00,2025-10-04 13:59:36.917160+00:00,OK,,[],95f7d951b948784e,2979a98f3de7f64a15fc6248e82e9f61,Find the Taylor polynomial of order 3 for f(x)...,CHAIN,"import sympy\nfrom sympy import cos, series, S...","{'langgraph_node': 'write_calculus_code', 'lan...",Find the Taylor polynomial of order 3 for f(x)...,"import sympy\nfrom sympy import cos, series, S...",Code executed successfully\n\n**Standard Outpu...,The Taylor polynomial of order 3 for f(x) = co...,factual,The code calculates the Taylor polynomial of o...,readable,"First, analyze the question. The question is s...",correct,The question asks for the Taylor polynomial of...
3,write_calculus_code,CHAIN,cd979c8f682ca58b,2025-10-04 13:59:07.968526+00:00,2025-10-04 13:59:11.661038+00:00,OK,,[],4a5711326795cb3e,5f0fd2066a1d357c9be4cd724e3ddc70,Use Green's theorem to evaluate ∮C (y dx - x d...,CHAIN,"import sympy\nfrom sympy import integrate, sin...","{'langgraph_node': 'write_calculus_code', 'lan...",Use Green's theorem to evaluate ∮C (y dx - x d...,"import sympy\nfrom sympy import integrate, sin...",Code executed successfully\n\n**Standard Outpu...,The user asked to evaluate the line integral ∮...,factual,The question asks to evaluate a line integral ...,readable,The code is well-structured and uses meaningfu...,correct,Explanation:\nThe question asks to evaluate th...
4,write_calculus_code,CHAIN,b4a67a0cafb0969f,2025-10-04 13:58:26.498836+00:00,2025-10-04 13:58:45.578624+00:00,OK,,[],df3ca0c226116a46,540176bfb532de742bc2696ec49a9f69,"Evaluate the line integral ∫C (x dx + y dy), w...",CHAIN,\nimport sympy\n\nx = sympy.Symbol('x')\ny = s...,"{'langgraph_node': 'write_calculus_code', 'lan...","Evaluate the line integral ∫C (x dx + y dy), w...",\nimport sympy\n\nx = sympy.Symbol('x')\ny = s...,Code executed successfully\n\n**Standard Outpu...,"The line integral ∫C (x dx + y dy), where C is...",factual,The code calculates the line integral of x dx ...,readable,The code uses the `sympy` library to calculate...,correct,Explanation:\nThe question asks to evaluate th...


## Logging the Evaluations

In [41]:
Client().spans.log_span_annotations_dataframe(
    dataframe=spanseval,
    annotation_name="Hallucination/Readability/ToolCall",
    annotator_kind="LLM",
)

In [36]:
from phoenix.client import AsyncClient
from phoenix.evals.utils import to_annotation_dataframe

client = AsyncClient()

annotations = to_annotation_dataframe(
    spanseval
)  # can also specify score_names to log only certain scores
await client.spans.log_span_annotations_dataframe(dataframe=annotations)

ImportError: cannot import name 'to_annotation_dataframe' from 'phoenix.evals.utils' (/Users/user1/AI/AI_Tutorial/GenAI_Tutorial/.venv/lib/python3.12/site-packages/phoenix/evals/utils.py)

In [22]:
Client().spans.log_span_annotations_dataframe(
    dataframe=spans_df[:10],
    annotation_name="Spans",
    annotator_kind="LLM",
)
Client().spans.log_span_annotations_dataframe(
    dataframe=hallucination_classifications[['label', 'hallucination_explanation']],
    annotation_name="Hallucination",
    annotator_kind="LLM",
)
Client().spans.log_document_annotations_dataframe(
    dataframe=code_readability[['label', 'code_readability_explanation']],
    annotation_name="Code Readability",
    annotator_kind="LLM",
)
Client().spans.log_span_annotations_dataframe(
    dataframe=tool_call_evaluations[['label', 'tool_call_explanation']],
    annotation_name="Tool Call Correctness",
    annotator_kind="LLM",
)

ValueError: DataFrame must contain at least one of: label, score, explanation